In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
data_fake = pd.read_csv("Fake.csv")
data_true = pd.read_csv("True.csv")

In [ ]:
data_fake.head()


In [ ]:
data_true.head()

In [ ]:
data_fake["class"]=0
data_true["class"]=1

In [ ]:
data_fake.shape,data_true.shape

In [ ]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
  data_fake.drop([i], axis=0 ,inplace = True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
  data_true.drop([i], axis=0 ,inplace = True)

In [ ]:
data_fake.shape,data_true.shape


In [ ]:
data_fake_manual_testing["class"] = 0
data_true_manual_testing["class"] = 1


In [ ]:
data_fake_manual_testing.head(10)

In [ ]:
data_true_manual_testing.head(10)

In [ ]:
data_merge = pd.concat([data_fake,data_true] , axis = 0)
data_merge.head(10)

In [ ]:
data_merge.columns

In [ ]:
data = data_merge.drop(["title","subject","date"] ,axis = 1)

In [ ]:
data.isnull().sum()

In [ ]:
data = data.sample(frac = 1)

In [ ]:
data.head()

In [ ]:
data.reset_index(inplace = True)
data.drop(["index"] , axis =1 , inplace = True)

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
def wordopt(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '',text)
  text = re.sub("\\W", " ", text)
  text = re.sub('https?://\S+|www.\.\S+' , '',text)
  text = re.sub('<.*?>+','',text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '',text)
  text = re.sub('\n','',text)
  text = re.sub('\w*\d\w' , '',text)
  return text

In [ ]:
data["text"] = data["text"].apply(wordopt)

In [ ]:
x = data["text"]
y = data["class"]

In [ ]:
x_train, x_test, y_train ,y_test = train_test_split(x,y, test_size =0.25)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)


In [ ]:
from sklearn .linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

In [ ]:
pred_lr = LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test, pred_lr))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

In [ ]:
print(classification_report(y_test,pred_dt))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

In [ ]:
pred_gb = GB.predict(xv_test)

In [ ]:
GB.score(xv_test , y_test)

In [ ]:
print(classification_report(y_test, pred_gb))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

In [ ]:
pred_rf = RF .predict(xv_test)

In [ ]:
RF.score(xv_test , y_test)

In [ ]:
print(classification_report(y_test ,pred_rf))

In [ ]:
def output_lable(n):
  if n==0:
    return "Fake News"
  elif n==1:
    return "Not A Fake News"

def manual_testing(news):
    new_def_test = pd.DataFrame({"text":[news]})
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_xv_test = vectorization.transform(new_def_test["text"])

    models = {
        "Logistic Regression": LR,
        "Decision Tree": DT,
        "Gradient Boosting": GB,
        "Random Forest": RF
    }

    for name, model in models.items():
        pred = model.predict(new_xv_test)[0]
        prob = model.predict_proba(new_xv_test)
        print(f"{name}: {output_lable(pred)} | Confidence: {max(prob[0])*100:.2f}%")


In [ ]:
news = str(input())
manual_testing(news)
